In [9]:
import json

with open('data/myip.json') as f:
    myip_domains_dict = json.load(f)

In [10]:
domains = myip_domains_dict['list']
urls = ['http://'+domain for domain in domains]

In [66]:
import panel as pn
pn.extension()

In [12]:
import spacy
from spacy.lang.ru.examples import sentences 

In [93]:
import requests
myip = requests.get('https://www.wikipedia.org').headers['X-Client-IP']
myip

'91.132.107.45'

In [96]:
r = requests.get(urls[1])

In [99]:
r.content.decode()

'91.132.107.45'

In [14]:
# pip install spacy

In [15]:
import asyncio
from pyppeteer import launch
from pyppeteer_stealth import stealth

proxy_url = 'zproxy.lum-superproxy.io:22225'
proxy_username = 'lum-customer-c_6666ddaf-zone-zone2'
proxy_password = 'yzkwtvfegvqc'
# browser = await launch(headless=True)
# Запуск браузера с помощью Pyppeteer
browser = await launch(headless=True, args=[
    '--enable-features=NetworkService', 
    f'--proxy-server={proxy_url}',
    '--no-sandbox',
      '--headless',
      '--disable-gpu',
      '--window-size=1920x1080',
])

async def scrape_page(url):
    # Запуск браузера с помощью Pyppeteer
    page = await browser.newPage()
    await page.authenticate({'username': proxy_username, 'password': proxy_password})

    # Использование Pyppeteer Stealth для обхода блокировок
    await stealth(page)

    # Загрузка страницы
    try:
        await page.goto(url)
    except Exception as ex:
        print(ex)
        return ''
#     html = await page.content()
    try:
        html = await page.evaluate('document.body.textContent', force_expr=True)
    except Exception as ex:
        print(ex)
        return ''
        
    await page.close()
    return html

In [16]:
from tqdm.auto import tqdm
async def grab_urls():
    htmls = []
    async for url in tqdm(urls):
        html = await scrape_page(url)
        htmls.append(html)
    return htmls 

In [17]:
htmls = await grab_urls()

  0%|          | 0/232 [00:00<?, ?it/s]

net::ERR_CONNECTION_REFUSED at http://api.wipmania.com
net::ERR_CONNECTION_RESET at http://bearsmyip.com
net::ERR_NAME_NOT_RESOLVED at http://bot.whatismyipaddress.com
net::ERR_NAME_NOT_RESOLVED at http://checkip-waw.dyndns.com
net::ERR_CERT_DATE_INVALID at http://checkip.narak.com
Navigation Timeout Exceeded: 30000 ms exceeded.
net::ERR_CONNECTION_TIMED_OUT at http://curlmyip.com
net::ERR_CONNECTION_REFUSED at http://dpool.sina.com.cn
net::ERR_NAME_NOT_RESOLVED at http://findmyipaddress.com


Future exception was never retrieved
future: <Future finished exception=NetworkError('Protocol error Target.sendMessageToTarget: Target closed.')>
pyppeteer.errors.NetworkError: Protocol error Target.sendMessageToTarget: Target closed.


net::ERR_NAME_NOT_RESOLVED at http://ip-detect.net
net::ERR_CERT_DATE_INVALID at http://ip-info.org
net::ERR_NAME_NOT_RESOLVED at http://ip.cctv.pk
net::ERR_CONNECTION_RESET at http://ip.my-proxy.com
net::ERR_CONNECTION_TIMED_OUT at http://ip.tool.la
net::ERR_CONNECTION_RESET at http://ip4.me
net::ERR_NAME_NOT_RESOLVED at http://ipaddress.org
net::ERR_NAME_NOT_RESOLVED at http://ipv4bot.whatismyipaddress.com
Navigation Timeout Exceeded: 30000 ms exceeded.
net::ERR_NAME_NOT_RESOLVED at http://ipv6bot.whatismyipaddress.com
net::ERR_CONNECTION_TIMED_OUT at http://mioip.ch
net::ERR_NAME_NOT_RESOLVED at http://mioip.org
net::ERR_NAME_NOT_RESOLVED at http://mioip.win
net::ERR_CONNECTION_TIMED_OUT at http://myip.by
net::ERR_NAME_NOT_RESOLVED at http://myip.cf
Navigation Timeout Exceeded: 30000 ms exceeded.
net::ERR_NAME_NOT_RESOLVED at http://myip.co.nz
Execution context was destroyed, most likely because of a navigation.
net::ERR_NAME_NOT_RESOLVED at http://myip.dnsdynamic.org
net::ERR_CONNE

In [35]:
htmls = [h for h in htmls if h]
len(htmls)

176

In [21]:
await browser.close()

In [84]:
len(htmls)

176

In [82]:
sum('185.163.156.81' in html for html in htmls)

105

In [85]:
import socket
hostname = socket.gethostname()
IPAddr = socket.gethostbyname(hostname)
 
print("Your Computer Name is:" + hostname)
print("Your Computer IP Address is:" + IPAddr)

Your Computer Name is:NB0820.local
Your Computer IP Address is:192.168.160.201


In [80]:
panel_html = pn.pane.HTML(htmls[10])
panel_html

HTML(str)

In [91]:
htmls[10]

'Current IP Address: 185.163.156.81\n'

In [86]:
from transformers import BertTokenizer, BertModel
from bs4 import BeautifulSoup

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained('bert-base-multilingual-cased')

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [24]:
# pip install torch torchvision torchaudio

In [58]:
import torch

def tokenize(text):
    # Токенизация текста
    tokens = tokenizer.tokenize(text)

    # Добавление токенов [CLS] и [SEP]
    tokens = ['[CLS]'] + tokens + ['[SEP]']

    # Преобразование токенов в идентификаторы
    input_ids = tokenizer.convert_tokens_to_ids(tokens)

    # Преобразование идентификаторов в тензор
    input_ids = torch.tensor(input_ids).unsqueeze(0)
    print(input_ids.shape)

    # Получение эмбеддингов
    with torch.no_grad():
        outputs = model(input_ids[:, :512])
        embeddings = outputs[0]
    return embeddings

In [59]:
# htmls[37]

In [61]:
tokenize(htmls[37])

torch.Size([1, 693])


tensor([[[ 2.9535e-01, -3.2360e-01, -1.2383e-04,  ..., -2.2333e-01,
           1.3749e-01, -3.3716e-01],
         [-3.3033e-01, -1.4010e-01,  2.9786e-01,  ...,  1.4941e+00,
           2.8014e-02, -6.0962e-01],
         [ 1.2417e-01, -1.6194e-01, -4.5748e-01,  ..., -1.2477e-01,
          -8.3248e-01, -1.2814e+00],
         ...,
         [-6.2503e-01,  3.3888e-01,  4.2715e-01,  ...,  1.4629e-01,
           3.6824e-01, -1.0064e+00],
         [-9.8453e-02, -6.2286e-01,  8.8044e-02,  ...,  8.4658e-01,
          -6.3543e-03, -2.1361e-01],
         [ 3.5910e-01, -1.0762e+00, -4.0355e-01,  ..., -8.2076e-02,
           8.7886e-01, -6.3453e-01]]])